In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn import datasets, neighbors, metrics, tree, svm, preprocessing, model_selection, ensemble
from sklearn.model_selection import StratifiedKFold
from pprint import pprint


In [2]:
#%%time
#
## load labels
#labels = pd.read_csv('../data/Lauren/Labels.csv')
##labels.head() # to display the first 5 lines of loaded data
#
## load data
#df = pd.read_csv('../data/Lauren/500_PBMC_3p_LT_Chromium_X_metrics_summary.csv') # takes about 5min


In [3]:
#%%time
## filter out certain data
def gereral_data_filter(df, labels, filter_on, amount_higher_than):
    ## Filter info
    classes_count = labels.groupby(filter_on).count()
    classes_to_keep = list(classes_count[classes_count[classes_count.columns[-1]] >= amount_higher_than].index)
    keep_indices = labels[filter_on].isin(classes_to_keep)
    
    ## delete entries part of class that's too small, remove names column
    return (df[keep_indices].drop(columns=["Unnamed: 0"]), labels[keep_indices])

#df, labels = gereral_data_filter(df, labels, "cluster", 10)

In [4]:
%%time
df = pd.read_pickle("../data/Lauren/df.pkl")
labels = pd.read_pickle("../data/Lauren/labels.pkl")

CPU times: user 16 ms, sys: 2.48 s, total: 2.5 s
Wall time: 2.52 s


In [5]:
def train_test_linear_nn(df, labels, class_column_name): # todo: give linear classifier as argument
    # only keep the needed column
    drop_columns = filter(lambda col: col != class_column_name , labels.columns)
    labels = labels.drop(columns = drop_columns).values.ravel()
    
    # use this to split dataset in 2 parts, test and train
    skf = StratifiedKFold(n_splits=2, random_state=1337, shuffle=True)
    
    for train_index, test_index in skf.split(df, labels):
        #test_index, train_index = train_index, test_index 
        # get train and test set
        X_train, X_test = df.take(train_index), df.take(test_index)
        y_train, y_test = labels[train_index], labels[test_index]
        
        # 1vRest training
        print("TRAIN:", train_index, "TEST:", test_index)
        print("Start fitting")
        lin_clf = svm.LinearSVC()
        lin_clf.fit(X_train, y_train)
        
        # predicting
        print("Start predicting")
        y_pred_lin_clf = lin_clf.predict(X_test)
        
        # metrics
        #print("Calculating metrics")
        
        print(f"F1 macro-average: {metrics.f1_score(y_test, y_pred_lin_clf, average='macro')}")
        print(f"F1 weighted-average: {metrics.f1_score(y_test, y_pred_lin_clf, average='weighted')}")
        f1 = metrics.f1_score(y_test, y_pred_lin_clf, average=None)
        print(f"accuracy: {metrics.accuracy_score(y_test, y_pred_lin_clf)}")
        
        unique_labels_df = pd.DataFrame(pd.Series(y_test).unique())
        f1_df = pd.DataFrame(f1)
        print(pd.concat([unique_labels_df, f1_df], axis=1, keys=['dcluster', 'f1_per_dcluster']))
        
        # just do 1 iteration
        return (lin_clf, f1)
    
    
#drop_columns = filter(lambda col: col != "Class" , labels.columns)
#list(drop_columns)

In [8]:
def train_linear_nn(df, labels, class_column_name): # todo: give linear classifier as argument
    # only keep the needed column
    drop_columns = filter(lambda col: col != class_column_name , labels.columns)
    labels = labels.drop(columns = drop_columns).values.ravel()
     
    X_train, y_train = df, labels
        
    # 1vRest training
    print(f"Start training {class_column_name} entries with multiclass output: {pd.Series(y_train).unique()}")
    lin_clf = svm.LinearSVC()
    lin_clf.fit(X_train, y_train)
    
    return lin_clf
    

In [9]:
%%time
# as a test, train the first neural network to divide the data into Classes
(clf, all_f1) = train_test_linear_nn(df, labels, "Class")
all_f1

NameError: name 'train_test_linear_nn' is not defined

In [10]:
# make tree structure
class Node:
    def __init__(self, parent, class_name):
        self.parent = parent
        self.class_name = class_name
        
        self.clf = None
        self.children = dict() # dict die resultaat van clf linkt aan een nieuwe node (met clf)
        
    def __str__(self):
        if self.parent is None:
            return "Root"
        return f"(class_name: {self.class_name}, parent: {self.parent})"
    
    def __repr__(self):
        return self.__str__()

In [11]:
def filter_data_on_class_name(df, labels, class_name, class_column_name):
    keep_indices = labels[class_column_name] == class_name
    return (df[keep_indices], labels[keep_indices])

def train_hyr_nn(df, labels, node, parent_class=None, parent_class_column=None):
    # train neural net to classify input in the child classes
    
    # get the child_class_column
    if (parent_class is None or parent_class_column is None):
        child_class_column = labels.columns[0]
    else:
        # make data smaller: remove all entries that do not belong to the parent_class
        
        df, labels = filter_data_on_class_name(df, labels, parent_class, parent_class_column)
        
        # get child_class_column
        child_class_column_index = list(labels.columns).index(parent_class_column) + 1
        if child_class_column_index >= len(labels.columns):
            # we are at in a leaf of the hyr tree, there are no further child classes
            return None
        child_class_column = labels.columns[child_class_column_index]
    
    
    # neural net that further classifies entries
    unique_labels = pd.Series(labels[child_class_column]).unique()
    if len(unique_labels) == 1:
        # the subclass is the same as the parent class
        node.clf = None
    else:
        print()
        print(f"parent_class: {parent_class}")
        node.clf = train_linear_nn(df, labels, child_class_column)
    
    # recursive step
    for child_class in unique_labels: # todo: parallelize
        child_node = Node(node, child_class)
        train_hyr_nn(df, labels, child_node, child_class, child_class_column)
        node.children[child_class] = child_node


In [186]:
%%time
# get part of data where all cluster types are represented

drop_columns = list(filter(lambda col: col != "cluster" , labels.columns))
dclusters = labels.drop(columns = drop_columns).values.ravel()

# only use part (1/5) of data for training and 1/5th for testing
trained = False
skf = StratifiedKFold(n_splits=2, random_state=1337, shuffle=True)
for train_index, test_index in skf.split(df, dclusters):
    if not trained:
        print("TRAINING")
        root = Node(None, "")
        train_hyr_nn(df.take(test_index), labels.take(test_index), root)
        trained = True
    else: 
        print("SETTING TEST SET")
        X_test_set = df.take(test_index)
        y_test_set = labels.take(test_index)
        break

root

TRAINING

parent_class: None
Start training Class entries with multiclass output: ['GABAergic' 'Glutamatergic' 'Non-Neuronal']

parent_class: GABAergic
Start training Subclass entries with multiclass output: ['Lamp5' 'Sst' 'Vip' 'Sncg' 'Serpinf1' 'Pvalb']


/home/iwijn/ml/machine-learning-project/venv/lib/python3.9/site-packages/sklearn/svm/_base.py:1199: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(



parent_class: Lamp5
Start training cluster entries with multiclass output: ['Lamp5 Lsp1' 'Lamp5 Krt73' 'Lamp5 Plch2 Dock5' 'Lamp5 Fam19a1 Tmem182'
 'Lamp5 Ntn1 Npy2r' 'Lamp5 Lhx6' 'Lamp5 Fam19a1 Pax6']

parent_class: Sst
Start training cluster entries with multiclass output: ['Sst Tac1 Tacr3' 'Sst Chrna2 Glra3' 'Sst Hpse Sema3c' 'Sst Mme Fam114a1'
 'Sst Nr2f2 Necab1' 'Sst Chodl' 'Sst Myh8 Etv1 ' 'Sst Myh8 Fibin'
 'Sst Chrna2 Ptgdr' 'Sst Crh 4930553C11Rik ' 'Sst Calb2 Pdlim5'
 'Sst Tac1 Htr1d' 'Sst Hpse Cbln4' 'Sst Crhr2 Efemp1' 'Sst Tac2 Tacstd2'
 'Sst Rxfp1 Prdm8' 'Sst Rxfp1 Eya1' 'Sst Esm1' 'Sst Tac2 Myh4' 'Sst Nts'
 'Sst Calb2 Necab1']


/home/iwijn/ml/machine-learning-project/venv/lib/python3.9/site-packages/sklearn/svm/_base.py:1199: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(



parent_class: Vip
Start training cluster entries with multiclass output: ['Vip Igfbp6 Car10' 'Vip Lect1 Oxtr' 'Vip Igfbp6 Pltp'
 'Vip Crispld2 Kcne4' 'Vip Pygm C1ql1' 'Vip Ptprt Pkp2'
 'Vip Arhgap36 Hmcn1' 'Vip Igfbp4 Mab21l1' 'Vip Crispld2 Htr2c'
 'Vip Lmo1 Fam159b' 'Vip Chat Htr1f' 'Vip Lmo1 Myl1' 'Vip Gpc3 Slc18a3'
 'Vip Rspo4 Rxfp1 Chat' 'Vip Rspo1 Itga4' 'Vip Col15a1 Pde1a']


/home/iwijn/ml/machine-learning-project/venv/lib/python3.9/site-packages/sklearn/svm/_base.py:1199: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(



parent_class: Sncg
Start training cluster entries with multiclass output: ['Sncg Vip Itih5' 'Sncg Slc17a8' 'Sncg Vip Nptx2' 'Sncg Gpr50']

parent_class: Serpinf1
Start training cluster entries with multiclass output: ['Serpinf1 Aqp5 Vip' 'Serpinf1 Clrn1']

parent_class: Pvalb
Start training cluster entries with multiclass output: ['Pvalb Reln Tac1' 'Pvalb Tpbg' 'Pvalb Gpr149 Islr' 'Pvalb Th Sst'
 'Pvalb Akr1c18 Ntf3' 'Pvalb Calb1 Sst' 'Pvalb Reln Itm2a' 'Pvalb Gabrg1'
 'Pvalb Vipr2' 'Pvalb Sema3e Kank4']

parent_class: Glutamatergic
Start training Subclass entries with multiclass output: ['L6 CT' 'L6b' 'L6 IT' 'L2/3 IT' 'L5 PT' 'L4' 'NP' 'L5 IT']

parent_class: L6 CT
Start training cluster entries with multiclass output: ['L6 CT VISp Krt80 Sla' 'L6 CT VISp Ctxn3 Sla' 'L6 CT VISp Ctxn3 Brinp3'
 'L6 CT VISp Gpr139' 'L6 CT VISp Nxph2 Wls' 'L6 CT ALM Nxph2 Sla']

parent_class: L6b
Start training cluster entries with multiclass output: ['L6b VISp Col8a1 Rprm' 'L6b VISp Mup5' 'L6b P2ry12'
 

/home/iwijn/ml/machine-learning-project/venv/lib/python3.9/site-packages/sklearn/svm/_base.py:1199: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(



parent_class: NP
Start training cluster entries with multiclass output: ['L5 NP VISp Trhr Cpne7' 'L5 NP VISp Trhr Met']

parent_class: L5 IT
Start training cluster entries with multiclass output: ['L5 IT VISp Hsd11b1 Endou' 'L5 IT VISp Batf3' 'L5 IT VISp Col27a1'
 'L5 IT VISp Col6a1 Fezf2' 'L5 IT VISp Whrn Tox2']
SETTING TEST SET
CPU times: user 3min 22s, sys: 31.1 s, total: 3min 53s
Wall time: 3min 53s


Root

In [179]:
root.children["Non-Neuronal"].children["Astro"].children["Astro Aqp4"]

(class_name: Astro Aqp4, parent: (class_name: Astro, parent: (class_name: Non-Neuronal, parent: Root)))

In [59]:
# pprint(root.children)
# pprint(root.children["GABAergic"].children)
pprint(root.children["GABAergic"].children["Lamp5"].children)
pprint(root.children["GABAergic"].children["Lamp5"].clf)

{'Lamp5 Fam19a1 Pax6': (class_name: Lamp5 Fam19a1 Pax6, parent: (class_name: Lamp5, parent: (class_name: GABAergic, parent: Root))),
 'Lamp5 Fam19a1 Tmem182': (class_name: Lamp5 Fam19a1 Tmem182, parent: (class_name: Lamp5, parent: (class_name: GABAergic, parent: Root))),
 'Lamp5 Krt73': (class_name: Lamp5 Krt73, parent: (class_name: Lamp5, parent: (class_name: GABAergic, parent: Root))),
 'Lamp5 Lhx6': (class_name: Lamp5 Lhx6, parent: (class_name: Lamp5, parent: (class_name: GABAergic, parent: Root))),
 'Lamp5 Lsp1': (class_name: Lamp5 Lsp1, parent: (class_name: Lamp5, parent: (class_name: GABAergic, parent: Root))),
 'Lamp5 Ntn1 Npy2r': (class_name: Lamp5 Ntn1 Npy2r, parent: (class_name: Lamp5, parent: (class_name: GABAergic, parent: Root))),
 'Lamp5 Plch2 Dock5': (class_name: Lamp5 Plch2 Dock5, parent: (class_name: Lamp5, parent: (class_name: GABAergic, parent: Root)))}
LinearSVC()


In [61]:
# This is how 1 branch of the hyr tree, all the way down, should be trained

# Train on the whole dataset to divide in Class
l1_res_df = pd.DataFrame(root.clf.predict(X_test_set))
l1_res_df.index = X_test_set.index # set the same indexes before further filtering
pprint(l1_res_df) # aha! this keeps the old indices

# filter out all entries that gave Class "GABAergic" and train on this subset to determine Subclass
l2_input = X_test_set[l1_res_df[0] == "GABAergic"]
l2_res_df = pd.DataFrame(root.children["GABAergic"].clf.predict(l2_input))
l2_res_df.index = l2_input.index
pprint(l2_res_df) # aha! this keeps the old indices

# filter out all entries that gave Subclass "Lamp5" and finally train on cluster
l3_input = l2_input[l2_res_df[0] == "Lamp5"]
l3_res_df = pd.DataFrame(root.children["GABAergic"].children["Lamp5"].clf.predict(l3_input))
l3_res_df.index = l3_input.index
pprint(l3_res_df) # aha! this keeps the old indices

                   0
7          GABAergic
10         GABAergic
12         GABAergic
16         GABAergic
18         GABAergic
...              ...
12818  Glutamatergic
12819      GABAergic
12820      GABAergic
12824  Glutamatergic
12825  Glutamatergic

[2556 rows x 1 columns]
           0
7      Lamp5
10       Vip
12       Vip
16     Lamp5
18     Lamp5
...      ...
12799    Sst
12803    Sst
12810  Pvalb
12819    Sst
12820    Sst

[1129 rows x 1 columns]
                       0
7            Lamp5 Krt73
16            Lamp5 Lsp1
18            Lamp5 Lsp1
38      Lamp5 Ntn1 Npy2r
56            Lamp5 Lsp1
...                  ...
12471   Lamp5 Ntn1 Npy2r
12472   Lamp5 Ntn1 Npy2r
12475         Lamp5 Lsp1
12491  Lamp5 Plch2 Dock5
12495  Lamp5 Plch2 Dock5

[218 rows x 1 columns]


In [116]:
# Train on the whole dataset to divide in Class
l1_res_df = pd.DataFrame(root.clf.predict(X_test_set))
l1_res_df.index = X_test_set.index # set the same indexes before further filtering
pprint(l1_res_df) # aha! this keeps the old indices

# filter out all entries that gave Class "GABAergic" and train on this subset to determine Subclass
l2_input = X_test_set[l1_res_df[0] == "Glutamatergic"]
l2_res_df = pd.DataFrame(root.children["Glutamatergic"].clf.predict(l2_input))
l2_res_df.index = l2_input.index
pprint(l2_res_df) # aha! this keeps the old indices

# Special case: everything in L4 class belongs to the same cluster (so no further predicting has to / can be done)
# filter out all entries that gave Subclass "Lamp5" and finally train on cluster
l3_input = l2_input[l2_res_df[0] == "L4"]
# instead of further classifying:
#l3_res_df = pd.DataFrame(root.children["Glutamatergic"].children["L4"].clf.predict(l3_input)) # clf is None
#l3_res_df.index = l3_input.index
# we set the result to the the more specific cluster name of the one and only child
l3_res_df = pd.DataFrame(index=l3_input.index, columns=[0]).fillna(root.children["Glutamatergic"].children["L4"].children["L4 IT VISp Rspo1"].class_name)

pprint(l3_res_df) # aha! this keeps the old indices

                   0
7          GABAergic
10         GABAergic
12         GABAergic
16         GABAergic
18         GABAergic
...              ...
12818  Glutamatergic
12819      GABAergic
12820      GABAergic
12824  Glutamatergic
12825  Glutamatergic

[2556 rows x 1 columns]
           0
113    L6 CT
118    L6 CT
119    L6 CT
124    L6 CT
126    L6 CT
...      ...
12800  L6 IT
12809  L6 IT
12818  L5 PT
12824  L5 PT
12825  L5 PT

[1426 rows x 1 columns]
                      0
480    L4 IT VISp Rspo1
484    L4 IT VISp Rspo1
486    L4 IT VISp Rspo1
489    L4 IT VISp Rspo1
491    L4 IT VISp Rspo1
...                 ...
11218  L4 IT VISp Rspo1
11220  L4 IT VISp Rspo1
11245  L4 IT VISp Rspo1
12447  L4 IT VISp Rspo1
12448  L4 IT VISp Rspo1

[267 rows x 1 columns]


In [187]:
# given the hyr nn tree and an input, predict the cluster

# recursive
def predict(node, X_test):
    #### Printing
    spaces = 1
    it_node = node
    while it_node.parent is not None:
        it_node = it_node.parent
        spaces += 2
    print((spaces*"--") + f"{node.class_name if node.parent is not None else 'Root' }")
    ####
    
    # the tree goes further down, but there is only 1 subclass and thus no further classifier needs to be executed
    if node.clf is None:
        child_node = list(node.children.values())[0]
        y_test = pd.DataFrame(index=X_test.index, columns=[0]).fillna(child_node.class_name) 
        #print(child_node.class_name)
    else:
        y_test = pd.DataFrame(node.clf.predict(X_test))
        y_test.index = X_test.index # keep original indices
    
    # we are in a leaf when the children dont have any children themselves
    # (We dont need to call predict on a child if they wont be able to futher classify to their children
    if list(node.children.values())[0].children == {}:
        return y_test
    else:
        # the children do have a clf to further classify, so further classify
        predictions = []
        for label, child_node in node.children.items():
            new_X_test = X_test[y_test[0] == label]
            predictions.append(predict(child_node, new_X_test))
        return pd.concat(predictions)
    
    

In [188]:
%%time 
y_pred_hyr = predict(root, X_test_set).sort_index(ascending=True)

--Root
------GABAergic
----------Lamp5
----------Sst
----------Vip
----------Sncg
----------Serpinf1
----------Pvalb
------Glutamatergic
----------L6 CT
----------L6b
----------L6 IT
----------L2/3 IT
----------L5 PT
----------L4
----------NP
----------L5 IT
------Non-Neuronal
----------Astro
CPU times: user 22.8 s, sys: 40.3 s, total: 1min 3s
Wall time: 41.9 s


In [149]:
# y_pred_hyr.loc[pd.isna(y_pred_hyr[0]), :].index
# y_test_set.loc[pd.isna(y_test_set["cluster"]), :].index
print(pd.isna(y_test_set["cluster"]).any())
pd.isna(y_pred_hyr[0]).any()

False


False

In [182]:
test_set_bools = y_test_set["cluster"] == "Astro Aqp4"
pred_bools = y_pred_hyr[0] == "Astro Aqp4"

pprint(test_set_bools)
pprint(pred_bools)

print(test_set_bools.any())
print(pred_bools.any()) # no predictions of Astro Aqp4 were found! this explains why F1 is 0
                        # seems like they are all called "Astro"


7        False
10       False
12       False
16       False
18       False
         ...  
12818    False
12819    False
12820    False
12824    False
12825    False
Name: cluster, Length: 2556, dtype: bool
7        False
10       False
12       False
16       False
18       False
         ...  
12818    False
12819    False
12820    False
12824    False
12825    False
Name: 0, Length: 2556, dtype: bool
True
True


In [139]:
pprint(y_test_set["cluster"])
pprint(y_pred_hyr[0])

0        Vip Arhgap36 Hmcn1
3        Vip Crispld2 Htr2c
4         Lamp5 Plch2 Dock5
6            Sncg Vip Itih5
11       Vip Crispld2 Kcne4
                ...        
12826       Sst Hpse Sema3c
12827     L5 PT VISp Chrna6
12828            Sncg Gpr50
12829     Pvalb Gpr149 Islr
12831      Sst Calb2 Pdlim5
Name: cluster, Length: 6390, dtype: object
0        Vip Arhgap36 Hmcn1
3        Vip Crispld2 Htr2c
4         Lamp5 Plch2 Dock5
6            Sncg Vip Itih5
11       Vip Crispld2 Kcne4
                ...        
12826       Sst Hpse Sema3c
12827     L5 PT VISp Chrna6
12828            Sncg Gpr50
12829     Pvalb Gpr149 Islr
12831      Sst Calb2 Pdlim5
Name: 0, Length: 6390, dtype: object


In [192]:
f1 = metrics.f1_score(y_test_set["cluster"], y_pred_hyr[0], average=None, labels = unique_labels_df[0])
acc = metrics.accuracy_score(y_test_set["cluster"], y_pred_hyr[0])

unique_labels_df = pd.DataFrame(pd.Series(y_test_set["cluster"]).unique())

f1_df = pd.DataFrame(f1)
f1_df_labeled = pd.concat([unique_labels_df, f1_df], axis=1, keys=['dcluster', 'f1_per_dcluster'])
print(f1_df_labeled)
print(f"acc: {acc}")
print()
print(f"F1 micro-average: {metrics.f1_score(y_test_set['cluster'], y_pred_hyr[0], average='micro')}")
print(f"F1 macro-average: {metrics.f1_score(y_test_set['cluster'], y_pred_hyr[0], average='macro')}")
print(f"F1 weighted-average: {metrics.f1_score(y_test_set['cluster'], y_pred_hyr[0], average='weighted')}")

              dcluster f1_per_dcluster
                     0               0
0   Vip Arhgap36 Hmcn1        0.835165
1   Vip Crispld2 Htr2c        0.837209
2    Lamp5 Plch2 Dock5        0.884354
3       Sncg Vip Itih5        0.823529
4   Vip Crispld2 Kcne4        0.884615
..                 ...             ...
88     L5 PT VISp Lgr5        0.736842
89    Sst Calb2 Necab1        0.315789
90          L6b P2ry12        0.941176
91       L6b VISp Mup5        0.981818
92        L6b VISp Crh        0.923077

[93 rows x 2 columns]
acc: 0.9020344287949922

F1 micro-average: 0.9020344287949922
F1 macro-average: 0.8420932014001663
F1 weighted-average: 0.9003196202455704


In [193]:
# export f1 scores (file in results/ folder)
f1_df_labeled.to_csv("all_f1_scores_hyr.csv", index=False)